In [ ]:
# default_exp tokenizer

In [ ]:
from nbdev.showdoc import *

# Tokenizer

> We are going to tokenize using 2 different strategies. The first one is using stopwords (read the main README for more information). And the second one, is a nltk grammar regexp parser.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from keywords2vec.imports import *

In [ ]:
#export
NUMBERS_STOPWORDS = {
    "en": [
        "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen", "twenty", "twenty-one", "twenty-two", "twenty-three", "twenty-four", "twenty-five", "twenty-six", "twenty-seven", "twenty-eight", "twenty-nine", "thirty", "thirty-one", "thirty-two", "thirty-three", "thirty-four", "thirty-five", "thirty-six", "thirty-seven", "thirty-eight", "thirty-nine", "forty", "forty-one", "forty-two", "forty-three", "forty-four", "forty-five", "forty-six", "forty-seven", "forty-eight", "forty-nine", "fifty", "fifty-one", "fifty-two", "fifty-three", "fifty-four", "fifty-five", "fifty-six", "fifty-seven", "fifty-eight", "fifty-nine", "sixty", "sixty-one", "sixty-two", "sixty-three", "sixty-four", "sixty-five", "sixty-six", "sixty-seven", "sixty-eight", "sixty-nine", "seventy", "seventy-one", "seventy-two", "seventy-three", "seventy-four", "seventy-five", "seventy-six", "seventy-seven", "seventy-eight", "seventy-nine", "eighty", "eighty-one", "eighty-two", "eighty-three", "eighty-four", "eighty-five", "eighty-six", "eighty-seven", "eighty-eight", "eighty-nine", "ninety", "ninety-one", "ninety-two", "ninety-three", "ninety-four", "ninety-five", "ninety-six", "ninety-seven", "ninety-eight", "ninety-nine"
    ],
    "es": []
}

In [ ]:
#export
def prepare_stopwords(stopwords=None, additional_stopwords=None, lang="en"):
    if stopwords is None:
        stopwords = safe_get_stop_words(lang) + (NUMBERS_STOPWORDS.get(lang) or [])
    if additional_stopwords:
        stopwords += additional_stopwords
    return [
        stopword
        for stopword in stopwords
        if stopword
    ]


def tokenize_one(text, stopwords=None, additional_stopwords=None, lang="en", split_by_stopwords=True):
    stopwords = prepare_stopwords(stopwords, additional_stopwords, lang)
    text_part = text.lower()

    regexs = []
    if split_by_stopwords:
        # Remove all stopwords by a !, we are searching for the stopword (bounded)
        regexs.append(
            ("\\b" + "\\b|\\b".join(stopwords), "!!")
        )
    # Must be executed in order
    regexs += [
        ("’", "'"),
        # Remove all non alpha, numeric, spaces, - or single quote
        (r'([^a-z0-9\u00C0-\u1FFF\u2C00-\uD7FF \n\-\'])', "!!"),
        # remove only words numbers
        (r'(^|[ !])[\-0-9]+([ !]|$)', "!!"),
        # remove hyphen-minus for keywords starting or ending with it
        (r'((^|[ !])[\-\']+)|([\-\']+([ !]|$))', "!!"),
        # remove spaces between !
        (r' *! *', "!!"),
        # generate multiple ! need for next regex
        (r'!', "!!"),
        # remove one character keyword
        (r'(^|!)[^!\n](!|$)', "!!"),
        # remove multiple ! (!!!!)
        (r'!+', "!"),
        # remove first and last !
        (r'(^!+)|(!+$)', "")
    ]

    for regex, replacement in regexs:
        text_part = re.sub(regex, replacement, text_part, flags=re.M)
    return text_part


# Second option to tokenize the information
def get_nodes_for_ntlk(parent, stopwords, valid_labels):
    keywords = []
    for node in parent:
        if type(node) is nltk.Tree:
            if node.label() in valid_labels:
                phrase = " ".join([key.lower() for key, value in node.leaves()])
                phrase = unidecode.unidecode(phrase)
                for subtree in node.subtrees():
                    subtree_keywords = get_nodes_for_ntlk(subtree, stopwords, valid_labels)
                    keywords.extend(subtree_keywords)
                if phrase not in stopwords:
                    pattern = re.compile(r"([^\s\w-]|_)+")
                    phrase = pattern.sub('', phrase).strip()
                    keywords.append(phrase)
    return keywords


def tokenize_by_nltk(text, stopwords=None, additional_stopwords=None, lang="en"):
    stopwords = prepare_stopwords(stopwords, additional_stopwords, lang)
    # grammar = r'KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}'
    grammar = r"""
        PHRASE1: {<NN.*>+ <DT>*}
        PHRASE2: {<JJ> <PHRASE1>+}
        PHRASE3: {<JJ>+ <PHRASE2>}
        PHRASE4: {(<JJ>* <NN.*>+ <IN>) <PHRASE3>}
    """
    valid_labels = ["PHRASE1", "PHRASE2", "PHRASE3", "PHRASE4", "KT"]

    chunker = nltk.RegexpParser(grammar, loop=5)
    chunker2 = nltk.RegexpParser(r"KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}", loop=5)
    output = ""

    for line in text.splitlines():
        sentences = nltk.sent_tokenize(line)
        sentences = [nltk.word_tokenize(sent) for sent in sentences]
        sentences = [nltk.pos_tag(sent) for sent in sentences]
        keyphrases = []
        relevant_words = []
        for sentence in sentences:
            keyphrases.append(chunker.parse(sentence))
            keyphrases.append(chunker2.parse(sentence))
        for elem in keyphrases:
            relevant_words += get_nodes_for_ntlk(elem, stopwords, valid_labels)
        output += "!".join(relevant_words) + "!"

    #output = re.sub("\\b" + "\\b|!".join(stopwords), "!", output, flags=re.M).lower()
    output = tokenize_one(output, split_by_stopwords=False)
    return output


def tokenize(text, text_output=False, lang="en", keywords_w_stopwords=False, merge=True):
    outputs = []
    tokenizers = [tokenize_one]
    if lang == "en" and keywords_w_stopwords:
        tokenizers.append(tokenize_by_nltk)

    for tokenizer_el in tokenizers:
        outputs.append(
            tokenizer_el(
                text,
                lang=lang
            )
        )
    if text_output:
        if merge:
            return "!".join(outputs)
        else:
            return outputs
    keywords = [
        [
            keyword.strip()
            for phrase in re.split("\r\n|\n", output)
            for keyword in phrase.split("!")
        ]
        for output in outputs
    ]
    if merge:
        return [item for sublist in keywords for item in sublist]
    else:
        return keywords

In [ ]:
show_doc(tokenize)

<h4 id="tokenize" class="doc_header"><code>tokenize</code><a href="__main__.py#L101" class="source_link" style="float:right">[source]</a></h4>

> <code>tokenize</code>(**`text`**, **`text_output`**=*`False`*, **`lang`**=*`'en'`*, **`keywords_w_stopwords`**=*`False`*, **`merge`**=*`True`*)



In [ ]:
#from_wikipedia
example_text = """The modern sovereign state of Chile is among South America's most economically and socially stable and prosperous nations, with a high-income economy and high living standards.[11][12] It leads Latin American nations in rankings of human development, competitiveness, income per capita, globalization, state of peace, economic freedom, and low perception of corruption.[13] It also ranks high regionally in sustainability of the state, and democratic development.[14] Currently it also has the lowest homicide rate in the Americas after Canada. Chile is a founding member of the United Nations, the Union of South American Nations (UNASUR), the Community of Latin American and Caribbean States (CELAC) and the Pacific Alliance, and joined the Organisation for Economic Co-operation and Development (OECD) in 2010."""
tokenized_text = tokenize(example_text)

Tokenizing the text, by default use two methods and merge them, so you might see duplicated keywords.

In [ ]:
tokenized_text[0:15]

['modern sovereign state',
 'chile',
 "among south america's",
 'economically',
 'socially stable',
 'prosperous nations',
 'high-income economy',
 'high living standards',
 'leads latin american nations',
 'rankings',
 'human development',
 'competitiveness',
 'income per capita',
 'globalization',
 'state']

If you wanted to how see the result from each method

In [ ]:
tokenized_text = tokenize(example_text, merge=False)

Tokenized using only stopwords

In [ ]:
tokenized_text[0][0:15]

['modern sovereign state',
 'chile',
 "among south america's",
 'economically',
 'socially stable',
 'prosperous nations',
 'high-income economy',
 'high living standards',
 'leads latin american nations',
 'rankings',
 'human development',
 'competitiveness',
 'income per capita',
 'globalization',
 'state']

Tokenized using only nltk, this method complement to find keywords with stopwords

In [ ]:
tokenized_text[1][0:15]

['sovereign state',
 'modern sovereign state',
 'chile',
 'south america',
 'nations',
 'prosperous nations',
 'economy',
 'high-income economy',
 'living standards',
 'high living standards',
 'modern sovereign state of chile',
 'south america',
 'prosperous nations',
 'high-income economy',
 'high living standards']

We recommend to use them with the default options

Tokenize and return plain text

In [ ]:
tokenized_text = tokenize(example_text, text_output=True)
tokenized_text

"modern sovereign state!chile!among south america's!economically!socially stable!prosperous nations!high-income economy!high living standards!leads latin american nations!rankings!human development!competitiveness!income per capita!globalization!state!peace!economic freedom!low perception!corruption!also ranks high regionally!sustainability!state!democratic development!currently!also!lowest homicide rate!americas!canada!chile!founding member!united nations!union!south american nations!unasur!community!latin american!caribbean states!celac!pacific alliance!joined!organisation!economic co-operation!development!oecd!sovereign state!modern sovereign state!chile!south america!nations!prosperous nations!economy!high-income economy!living standards!high living standards!modern sovereign state of chile!south america!prosperous nations!high-income economy!high living standards!nations!american nations!nations!latin american nations!rankings!development!human development!competitiveness!income!c